# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import os

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [6]:
#Produto escolhido:
produto = 'intel'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [8]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [9]:
print(os.getcwd())
dados = pd.ExcelFile('intel.xlsx',sep='.')
dados.sheet_names
Teste= dados.parse('Teste')
Treinamento = dados.parse('Treinamento')

C:\Users\rod_c\Documents\GitHub\ciencia_dados\Projeto 2


In [10]:
#eliminando para Teste

Teste = Teste.replace('#','',regex=True)
Teste = Teste.replace(',','',regex=True)
Teste = Teste.replace('()','',regex=True)
Teste = Teste.replace('@','',regex=True)
Teste = Teste.replace('rt','',regex=True)
Teste = Teste.replace(':','',regex=True)
Teste = Teste.replace('-','',regex=True)
Teste = Teste.replace('https','',regex=True)
Teste = Teste.replace('®','',regex=True)
Teste = Teste.replace('!','',regex=True)

In [11]:
#eliminando para treinamento

Treinamento = Treinamento.replace('#','',regex=True)
Treinamento = Treinamento.replace(',','',regex=True)
Treinamento = Treinamento.replace('()','',regex=True)
Treinamento = Treinamento.replace('@','',regex=True)
Treinamento = Treinamento.replace('rt','',regex=True)
Treinamento = Treinamento.replace(':','',regex=True)
Treinamento = Treinamento.replace('-','',regex=True)
Treinamento = Treinamento.replace('!','',regex=True)
#Treinamento = Treinamento.replace('\n','',regex=True)
Treinamento = Treinamento.replace('https','',regex=True)
Treinamento = Treinamento.replace('®','',regex=True)
#Treinamento = Treinamento.replace('.','',regex=True)



In [12]:
lista = []
R = 0
I = 0 
for values in Treinamento.Classificação:
    lista.append(values.split())
for i in lista:
    if i == ['relevante']:
        R +=1
    else:
        if i == ['irrelevante']:
            I +=1
#quantidade de irrelevantes
print(I)
#quantidade de relevantes
print(R)

#porcentagem de relevantes e irrelevanes
relevante = R/300
irrelevante = I/300
print(relevante)
print(irrelevante)

rev_ = Treinamento[Treinamento.Classificação == 'relevante']
irrev_ = Treinamento[Treinamento.Classificação == 'irrelevante']

190
110
0.36666666666666664
0.6333333333333333


In [13]:
#treinamento

treinamento_b = Treinamento.Treinamento
classe = Treinamento.Classificação
dict_pal = dict()
for x in range(0, len(treinamento_b)):
    situação = classe[x]
    twt= treinamento_b[x].split()
    for palavras in twt:
        if palavras in dict_pal:
            if situação == "relevante":
                dict_pal[palavras][0] += 1
            else:
                dict_pal[palavras][1] += 1
        else:
            if situação == 'relevante':
                dict_pal[palavras] = [1,0]
            else:
                dict_pal[palavras] = [0,1]
#print(dict_pal)
#print(len(dict_pal))


In [14]:
##Mostra a soma de todas as palavras dentro de relevante ou irrelevante

resultado_bom = 0
resultado_neutro = 0
for values in dict_pal:
    resultado_bom += dict_pal[values][0]
    resultado_neutro += dict_pal[values][1]
print(resultado_bom) #qunatidade de palavras relevante
print(resultado_neutro) #qunatidade de palavras irrelevantes

1767
3064


In [15]:
#porcentagem de cada palavra em relevante

coluna_r = np.sum(rev_.Treinamento + '').split()
coluna_r2 = pd.Series(coluna_r) 
# mostra uma tabela com a porcentagem de cada palavra em relevantes
#print(coluna_r2.value_counts(' ')*100)
# mostra uma tabela com a quantidade de cada palavra em relevantes
#print(coluna_r.value_counts(''))


#porcentagem para a palavra Intel dentro de relevantes
coluna_r.count('intel')*100/(resultado_bom)
#coluna = pd.Series(np.sum(Treinamento.Treinamento + '').split())
#coluna.value_counts('')

4.527447651386531

In [16]:
# para contar separadamente cada palavra a linha de codico à baixo apresenta erro
coluna_i =  pd.Series(np.sum(irrev_.Treinamento + '').split())
#porcentagem de cada palavra em irrelevante (necessario separqar em duas partes para contar as palavras separadamente)
coluna_i = np.sum(irrev_.Treinamento + '').split()
coluna_i2 = pd.Series(coluna_i)
#print(coluna_i2.value_counts(' ')*100)
col = len(coluna_i)

#porcentagem da palavra intel dentro de irrelevantes
coluna_i.count('intel')*100/(resultado_neutro)

4.5365535248041775

In [17]:
fr1 = "com"

# Função criada para sistematizar a classificação das frases, testados em um primeiro momento com apenas palavras

def leitura (x):
    frase = x.split(' ') # dividir a frase pelos espaços
    relevante = []
    irrelevante = [] 
    for e in range(len(frase)):

        relevante.append(((coluna_r.count(frase[e])))*100/(resultado_bom))
        irrelevante.append((coluna_i.count(frase[e]))*100/(resultado_neutro))
        
    print(relevante)
    print(irrelevante)
    probR = np.prod(relevante)
    probIR = np.prod(irrelevante)
    if probR > probIR:
        return "relevante"
    else:
        return "irrelevante"

leitura("com")

[2.320316921335597]
[0.97911227154047]


'relevante'

In [18]:
# Atribuindo todas as frases de arquivo execel Teste da coluna teste em uma lista, para futuramente utiliza-la na função Leitura
lis =[]
for values in Teste.Teste:
    lis.append(values)

# Mesma função utilizada no item anterior 
def leitura (x):
    frase = x.split(' ') # dividir a frase pelos espaços
    relevante = []
    irrelevante = []
    resultado=[]
    for e in range(len(frase)):

        relevante.append(((coluna_r.count(frase[e])))*100/(resultado_bom))
        irrelevante.append((coluna_i.count(frase[e]))*100/(resultado_neutro))
        
    probR = np.prod(relevante)
    probIR = np.prod(irrelevante)
    if probR > probIR:
        resultado.append("relevante")
        
    else:
        resultado.append("irrelevante")
    return resultado    

# Criação de outra lista para armazenar o que foi retornado da função para as frases dentro de Lis
computador=[]
for e in lis:
    x=leitura(e)
    computador.append(x)


# Código para contar a quantidade de irrelevantes e relevantes, mas não é necessário para os futuros resultados,
#portanto está comentado

#relevantes_v = 0
#irrelevantes_v = 0
#for e in lis:
#    if leitura(e) == "relevante":
#        relevaentes_v +=1
#    if leitura(e) == "irrelevante":
#        irrelevantes_v +=1
#    print(leitura(e))
#
#print(relevaentes_v) #qunatidade de palavras relevantes
#print(irrelevantes_v) #quantidade de palavras irrelevantes



    

In [19]:
#Obtivemos um problema com a célula anterior, na tabela a baixo, na coluna pc, as palavras apareciam entre chaves 
flat_list =[]
for sublist in abc:
    for item in sublist:
        flat_list.append(item)   
# criando uma nova coluna em que o próprio programa declarou se a mensagem era releveante ou irrelevaente
Teste['pc'] = pd.Series(flat_list)
Teste
# o código acima foi utilizado para solucionar esse problema

,Teste,Classificação,pc
0,notebook gamer acer vx 5 intel core i5 7ª gera...,irrelevante,irrelevante
1,lucains olha esse oq vc acha? //t.co/pla15u3gf9,irrelevante,irrelevante
2,gostei de um vídeo youtube //t.co/hvurmglk6p f...,irrelevante,irrelevante
3,sacolacheiax confira no magazine espacomall n...,irrelevante,irrelevante
4,samsung anuncia mega fábrica e passa intel com...,irrelevante,irrelevante
5,o intel atom era o processador do meu zenfone ...,irrelevante,irrelevante
6,se tem intel tem a máxima produtividade você ...,relevante,irrelevante
7,seus dados protegidos onde você estiver com de...,irrelevante,relevante
8,"10 acer es1132c9nx ordenador poátil de 11.6"" ...",irrelevante,irrelevante
9,pela primeira vez em mais de uma década a amd ...,relevante,irrelevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [20]:
br = []

for a in range(len(Teste.Teste)):
    if Teste.Classificação[a] == 'relevante':
        if Teste.pc[a] == 'relevante':
            br.append('positivos verdadeiros')
        if Teste.pc[a] == 'irrelevante':
            br.append('negativos falsos')
    elif Teste.Classificação[a] == 'irrelevante':
        if Teste.pc[a] == 'irrelevante':
            br.append('negativos verdadeiros')
        if Teste.pc[a] == 'relevante':
            br.append('positivos falsos')

##

#porcentagem das medidas:

##
porcentagem = pd.Series(br)
hue= (porcentagem.value_counts())/200
hue

negativos verdadeiros    0.75
negativos falsos         0.19
positivos falsos         0.05
positivos verdadeiros    0.01
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


Considerando os resultados obtidos podemos tirar algumas conclusões, tendo em vista a quantidade maior de Irrelevantes do que Relevantes, tanto em treinamento como em Teste, o numero de Negativos verdadeiros atingiu um acerto muito positivo. Porém ao analisarmos o que aconteceu no Treinamento, onde a a maioria das palavras era mais frequente em irrelevantes e que palavras específicas, por exemplo ótimo, horrível apareciam tão pouco que as frases para serem classificadas dependiam de outras menos representativas. Pensando no significado oposto aos das específicas, as palavras não representativas são as estão ambas representadas em irrelevantes como em relevantes, mas como na maioria dos casos são mais presentes em irrelevantes isso acarreta em uma chance bem maior do "classificador automático" dizer que as frases são irrelevantes. <br />

Ressaltando o que foi dito no parágrafo anterior, ao analisarmos as outras classificações e e suas porcentagens, negativos falsos 0.19%, positivos falsos 0.05%, positivos verdadeiros 0.01%, especialmente os negativos falsos e os positivos falsos observamos que 95% dois positivos classificados manualmente foram considerados negativos e apenas 6,66% dos negativos classificados manualmente foram considerados positivos. Isso apenas reforça o primeiro parágrafo indicando uma forte tendencia das frases serem classificadas como Irrelevantes. Porém isso não é um aspecto apenas negativos, pode estar fortemente atrelado e pequena quantidade de palavras pertencente fortemente a apenas a relevante, por exemplo. Para um futuro plano de expansão concluimos que estes 76% de acerto obtido podem aumentar com o aumento de tweets classificados manualmente, implicando diretamente na maior aparição destas palavras representativas aumentando ainda mais a oporcentagem de acertos e principalmente classificando corretamente cada vez mais frases relevantes.<br />

As mensagens de dupla negação e sarcasmos foram reconhecidas com a classificação manual, mas com a automática podem ter passado desapercebidas pois dependem apenas da probablidade das palavras que a compõem, para uma futura implementação considerar apenas essas palavras e calcular a probabilidade de cada palavra estarem dentro deste tipo de frase, mas para isso seria necessário uma amosta para obter um número significativo de duplo negativo e sarcasmo.<br />
     